In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install Faker

In [2]:
import re
import pandas as pd
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm
from faker import Faker
from collections import defaultdict

# Загрузка модели и токенизатора для русского языка
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Перемещение модели на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Генерация текста
# Изменение параметров генерации:
#     temperature: Управляет случайностью выборов модели. Более высокие значения
#     (например, 0.7, 1.0) приводят к большей случайности.
#     top_k: Ограничивает выбор следующего слова среди top_k самых вероятных слов.
#     Более низкие значения приводят к более разнообразным результатам.
#     top_p (nucleus sampling): Выбирает слова из вероятностной массы top_p.
#     Обычно используется значение между 0.8 и 0.9.

def generate_text(prompt, max_length=200, num_return_sequences=1,
                  num_beams=5, temperature=0.9, top_k=50, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Генерация текста с помощью модели
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        early_stopping=True,
        num_beams=num_beams,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
    )

    # Декодирование и возврат сгенерированных текстов
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


def only_letters(text):
    words = re.findall('[а-яё]+', text.lower())
    return ' '.join(words)


def generate_fake_words(num_texts=100, min_words=5, max_words=30):
    faker = Faker("ru_RU")
    fake_words = []
    for _ in range(num_texts):
        num_words = random.randint(min_words, max_words)
        fake_words.append(faker.words(num_words))

    return fake_words


prompt_templates = [
    "Напиши диалог, в котором клиент сомневается насчет покупки квартиры, и риэлтор его уговаривает.",
    "Напиши диалог, в котором клиент задает вопросы о районе, где находится квартира, а агент подробно рассказывает о его преимуществах.",
    "Напиши диалог, в котором клиент интересуется наличием парковочных мест, и агент рассказывает о вариантах парковки.",
    "Напиши диалог, в котором клиент выражает обеспокоенность по поводу ремонта в квартире, и агент предлагает решения.",
    "Напиши диалог, в котором клиент спрашивает о возможностях ипотеки, а агент подробно объясняет условия и возможности.",
    "Напиши диалог, в котором клиент интересуется, какие школы и детские сады находятся рядом с квартирой, и агент дает информацию.",
    "Напиши диалог, в котором клиент хочет узнать о коммунальных услугах и ежемесячных платежах, и агент предоставляет все данные.",
    "Напиши диалог, в котором клиент задает вопросы о безопасности района, и агент рассказывает о мерах безопасности и статистике.",
    "Напиши диалог, в котором клиент просит организовать осмотр квартиры, и агент договаривается о времени и дате.",
    "Напиши диалог, в котором клиент интересуется историей здания и соседями, а агент дает исчерпывающие ответы.",
    "Напиши диалог, в котором клиент спрашивает о процессе покупки квартиры, и агент объясняет все этапы.",
    "Напиши диалог, в котором клиент интересуется возможностью перепланировки квартиры, и агент рассказывает о правилах и ограничениях.",
    "Напиши диалог, в котором клиент выражает сомнения насчет качества строительства, и агент убеждает его в надежности здания.",
    "Напиши диалог, в котором клиент хочет узнать, какие магазины и услуги находятся в пешей доступности от квартиры, и агент дает полную информацию.",
    "Напиши диалог, в котором клиент спрашивает о предыдущих владельцах квартиры, и агент рассказывает историю объекта.",
    "Напиши диалог, в котором клиент интересуется наличием балкона или террасы, и агент описывает все внешние пространства.",
    "Напиши диалог, в котором клиент хочет узнать о возможности аренды квартиры, и агент рассказывает об условиях аренды.",
    "Напиши диалог, в котором клиент задает вопросы о транспортной доступности и ближайших остановках, и агент дает полную информацию.",
    "Напиши диалог, в котором клиент интересуется возможностью расширения или объединения квартиры с соседними, и агент объясняет возможные варианты.",
    "Напиши диалог, в котором клиент интересуется, сколько времени занимает процесс оформления покупки квартиры, и агент объясняет все детали.",
    "Напиши диалог, в котором клиент спрашивает о возможных налогах при покупке квартиры, и агент рассказывает о всех налоговых аспектах.",
    "Напиши диалог, в котором клиент интересуется качеством окон и дверей в квартире, и агент описывает их особенности.",
    "Напиши диалог, в котором клиент хочет узнать о шумоизоляции в квартире, и агент дает подробную информацию.",
    "Напиши диалог, в котором клиент спрашивает о возможности подключения к интернету и телевидению, и агент рассказывает о доступных провайдерах.",
    "Напиши диалог, в котором клиент интересуется планом технического обслуживания здания, и агент объясняет, как это организовано.",
    "Напиши диалог, в котором клиент хочет узнать о возможных проблемах с отоплением и кондиционированием, и агент рассказывает о состоянии систем.",
    "Напиши диалог, в котором клиент интересуется доступностью зеленых зон и парков рядом с квартирой, и агент рассказывает о ближайших парках.",
    "Напиши диалог, в котором клиент спрашивает о процессе регистрации права собственности, и агент объясняет, как это происходит.",
    "Напиши диалог, в котором клиент интересуется возможностью оформления договора с рассрочкой, и агент рассказывает об условиях.",
    "Напиши диалог, в котором клиент спрашивает о возможных ремонтах в подъезде или доме, и агент объясняет планы управляющей компании.",
    "Напиши диалог, в котором клиент интересуется возможностью переезда в квартиру в течение следующего месяца, и агент проверяет доступность.",
    "Напиши диалог, в котором клиент хочет узнать о наличии кладовых и подсобных помещений, и агент описывает их.",
    "Напиши диалог, в котором клиент интересуется соседями и культурой дома, и агент рассказывает о социальной среде.",
    "Напиши диалог, в котором клиент хочет узнать о возможных перепадах напряжения в электросети, и агент дает информацию.",
    "Напиши диалог, в котором клиент интересуется историей капитальных ремонтов в доме, и агент рассказывает о проведенных работах.",
    "Напиши диалог, в котором клиент спрашивает о доступности медицинских учреждений рядом с квартирой, и агент предоставляет информацию.",
    "Напиши диалог, в котором клиент интересуется возможностью установки дополнительных систем безопасности в квартире, и агент рассказывает о вариантах.",
    "Напиши диалог, в котором клиент спрашивает о средних коммунальных расходах по сезону, и агент предоставляет соответствующие данные.",
    "Напиши диалог, в котором клиент хочет узнать о правовых аспектах сделки, и агент объясняет, какие документы понадобятся и как их оформить.",

]

nums = ('один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь', 'девять',
        'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать', 'пятнадцать',
        'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
        'двадцать один', 'двадцать два', 'двадцать три', 'двадцать четыре',
        'двадцать пять', 'двадцать шесть', 'двадцать семь', 'двадцать восемь',
        'двадцать девять', 'тридцать'
        )

num_conv = ('одного', 'двух', 'трех', 'четырех', 'пяти', 'шести', 'семи', 'восьми', 'девяти',
            'десяти', 'одиннадцати', 'двенадцати', 'тринадцати', 'четырнадцати', 'пятнадцати',
            'шестнадцати', 'семнадцати', 'восемнадцати', 'девятнадцати', 'двадцати')

percents = ('процент', 'процента', 'процентов', 'проценты',)

approximates = ('порядка', 'около', 'примерно', 'почти', 'приблизительно', 'ориентировочно',
                'гдето', '')

discount = ("скидка",  # Именительный падеж
            "скидки",  # Родительный падеж
            "скидке",  # Дательный падеж
            "скидку",  # Винительный падеж
            "скидкой",  # Творительный падеж
            'скидок',
            'скидочка',
            )

rub_nums = ("сто тысяч",
            "двести тысяч",
            "триста тысяч",
            "пятьсот тысяч",
            "шестьсот тысяч",
            "миллион",
            "полтора миллиона",
            )

rub_text = ('рублей', 'рубли', 'руб', 'рубля', '')

num_texts = 5500

# Генерация нескольких текстов с различными скидками
generated_texts = []
for _ in tqdm(range(num_texts), total=num_texts):  # Генерация текстов
    prompt = random.choice(prompt_templates)
    generated_texts.extend(generate_text(prompt.replace('Напиши диалог, в котором ', ''),
                                         max_length=100 + random.randint(0, 400),
                                         num_return_sequences=1))

fake_words = generate_fake_words(num_texts=num_texts)

df = pd.DataFrame(columns=['processed_text', 'target_labels_positions'])

for idx_rec, (text, fake) in enumerate(zip(generated_texts, fake_words)):

    words = re.findall('[а-яё]+', text.lower())

    labels = defaultdict(list)

    # В каждую третью запись будем лепить скидки
    if not idx_rec % 3 or not idx_rec % 5 or not idx_rec % 7:

        if len(fake) < 10:
            words = fake + words
        else:
            idx = random.randint(5, len(fake) - 5) // 2
            words = fake[:idx] + words + fake[idx:]

        count_discnt = random.choice([1, 2])
        for nub_cnt in range(count_discnt):
            number = random.choice(nums + num_conv[:-1] + rub_nums)
            if number in rub_nums:
                prs = random.choice(rub_text)
            else:
                prs = random.choice(percents)
                if number in num_conv:
                    prs = 'процентов'
                    # тут формируются фразы вида: двух трех, пяти шести
                    idx_num = num_conv.index(number)
                    number = f'{number} {num_conv[idx_num + 1]}'
                    apx = random.choice(approximates)
                    if apx:
                        number = f'{apx} {number}'
                    if idx_num < 5:
                        prs = 'процента'

            number = f'{number} {prs}'.strip().split()  # процент скидки
            discnt = random.choice(discount)  # слово скидка
            place = random.choice([0, 1])  # расположения слова "скидка": 1 -> после процента
            diffs = random.choice([1, 2, 1, 3, 2, 1])  # смещение слова "скидка" от процентов

            # Первая итерация: начальная и конечная позиция скидки в тексте
            fst_pos = 3
            if not nub_cnt and count_discnt > 1:
                last_pos = (len(words) - 5) // 2
            else:
                if count_discnt > 1:
                    fst_pos = last_pos + 8
                last_pos = len(words) - len(number) - diffs - 2

            start_idx = random.randint(fst_pos, last_pos)
            disc_idx = start_idx + len(number) + diffs if place else start_idx - diffs

            labels['B-discount'].append(disc_idx)
            words[disc_idx] = discnt

            for i, txt_num in enumerate(number):
                if not i:
                    labels['B-value'].append(start_idx)
                else:
                    labels['I-value'].append(start_idx + i)
                words[start_idx + i] = txt_num

    text = ' '.join(words)

    df.loc[len(df)] = [text, dict(labels)]

df.to_csv('fake_text2.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.15s/it]


In [ ]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(f'*.csv')
zip_filename = 'fake_text2.zip'
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)